# Install the necessary requirements

In [10]:
!pip3 install aitoai==0.4.0

In [11]:
import pandas as pd
import numpy as np
from aito.schema import AitoStringType, AitoTextType, AitoDelimiterAnalyzerSchema, AitoTableSchema, AitoColumnLinkSchema, AitoDatabaseSchema
from aito.client import AitoClient
import aito.api as aito_api

In [12]:
from pathlib import Path
# Change the data folder to where you have downloaded the original data
data_folder = (Path('.') / 'data' / 'original').resolve()

# Load and process data files

## Places files

### chefmozaccepts.csv file

In [13]:
places_accept_df = pd.read_csv(data_folder / f'chefmozaccepts.csv')
places_accept_df

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards
...,...,...
1309,132012,Diners_Club
1310,132002,MasterCard-Eurocard
1311,132002,Visa
1312,132002,American_Express


Group the restaurant by its placeID since Aito supports multi values in a column

In [14]:
places_accept_df.rename(columns={'Rpayment': 'payment'}, inplace=True)
places_accept_df = places_accept_df.groupby('placeID').agg({'payment': lambda x: ';'.join(x)}).reset_index()
places_accept_df

,placeID,payment
0,132002,MasterCard-Eurocard;Visa;American_Express;Dine...
1,132012,Visa;American_Express;bank_debit_cards;Diners_...
2,132019,MasterCard-Eurocard;Visa;American_Express;Dine...
3,132023,American_Express;Japan_Credit_Bureau;bank_debi...
4,132024,Visa;American_Express;bank_debit_cards;Diners_...
...,...,...
610,135105,cash
611,135106,cash;VISA;MasterCard-Eurocard
612,135107,cash;VISA;MasterCard-Eurocard;American_Express...
613,135109,cash


### chefmozparking.csv file

In [15]:
places_parking_df = pd.read_csv(data_folder / f'chefmozparking.csv')
places_parking_df

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none
...,...,...
697,132024,street
698,132023,fee
699,132023,public
700,132012,public


In [16]:
places_parking_df = places_parking_df.groupby('placeID').agg({'parking_lot': lambda x: ';'.join(x)}).reset_index()
places_parking_df

,placeID,parking_lot
0,132012,public;street
1,132023,fee;public
2,132024,street
3,132026,fee
4,132030,street
...,...,...
670,135107,none
671,135108,none
672,135109,none
673,135110,none


### chefmozcuisine.csv file

In [17]:
places_cuisine_df = pd.read_csv(data_folder / f'chefmozcuisine.csv')
places_cuisine_df

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food
...,...,...
911,132005,Seafood
912,132004,Seafood
913,132003,International
914,132002,Seafood


In [18]:
places_cuisine_df.rename(columns={'Rcuisine': 'cuisine'}, inplace=True)
places_cuisine_df = places_cuisine_df.groupby('placeID').agg({'cuisine': lambda x: ';'.join(x)}).reset_index()
places_cuisine_df

,placeID,cuisine
0,132001,Dutch-Belgian
1,132002,Seafood
2,132003,International
3,132004,Seafood
4,132005,French;Seafood
...,...,...
764,135105,Fast_Food
765,135106,Mexican
766,135107,Latin_American
767,135109,Italian


### geoplaces2.csv file

In [19]:
places_geo_df = pd.read_csv(data_folder / f'geoplaces2.csv', encoding='latin-1')
places_geo_df

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincï¿½n de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,132866,22.141220,-100.931311,0101000020957F000013871838EC4A58C1B5DF74F8E396...,Chaires,Ricardo B. Anaya,San Luis Potosi,San Luis Potosi,Mexico,?,...,No_Alcohol_Served,not permitted,informal,completely,medium,?,familiar,f,closed,none
126,135072,22.149192,-101.002936,0101000020957F0000E7B79B1DB94758C1D29BC363D8AA...,Sushi Itto,Venustiano Carranza 1809 C Polanco,San Luis Potosi,SLP,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,sushi-itto.com.mx,familiar,f,closed,none
127,135109,18.921785,-99.235350,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,?,?,?,?,?,...,Wine-Beer,not permitted,informal,no_accessibility,medium,?,quiet,f,closed,Internet
128,135019,18.875011,-99.159422,0101000020957F0000B49B2E5C6E785AC12F9D58435241...,Restaurant Bar Coty y Pablo,Paseo de Las Fuentes 24 Pedregal de Las Fuentes,Jiutepec,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,closed,none


In [20]:
places_geo_df.replace('?', np.nan, inplace=True)  # replace the unknown value '?' with np.Nan
places_geo_df

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,NaN,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincï¿½n de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,NaN,...,Wine-Beer,only at bar,informal,partially,medium,NaN,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,NaN,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,NaN,...,No_Alcohol_Served,permitted,informal,completely,medium,NaN,familiar,t,closed,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,132866,22.141220,-100.931311,0101000020957F000013871838EC4A58C1B5DF74F8E396...,Chaires,Ricardo B. Anaya,San Luis Potosi,San Luis Potosi,Mexico,NaN,...,No_Alcohol_Served,not permitted,informal,completely,medium,NaN,familiar,f,closed,none
126,135072,22.149192,-101.002936,0101000020957F0000E7B79B1DB94758C1D29BC363D8AA...,Sushi Itto,Venustiano Carranza 1809 C Polanco,San Luis Potosi,SLP,Mexico,NaN,...,No_Alcohol_Served,none,informal,no_accessibility,medium,sushi-itto.com.mx,familiar,f,closed,none
127,135109,18.921785,-99.235350,0101000020957F0000A6BF695F136F5AC1DADF87B20556...,Paniroles,NaN,NaN,NaN,NaN,NaN,...,Wine-Beer,not permitted,informal,no_accessibility,medium,NaN,quiet,f,closed,Internet
128,135019,18.875011,-99.159422,0101000020957F0000B49B2E5C6E785AC12F9D58435241...,Restaurant Bar Coty y Pablo,Paseo de Las Fuentes 24 Pedregal de Las Fuentes,Jiutepec,Morelos,Mexico,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,f,closed,none


In [21]:
places_geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   placeID         130 non-null    int64  
 1   latitude        130 non-null    float64
 2   longitude       130 non-null    float64
 3   the_geom_meter  130 non-null    object 
 4   name            130 non-null    object 
 5   address         103 non-null    object 
 6   city            112 non-null    object 
 7   state           112 non-null    object 
 8   country         102 non-null    object 
 9   fax             0 non-null      float64
 10  zip             56 non-null     object 
 11  alcohol         130 non-null    object 
 12  smoking_area    130 non-null    object 
 13  dress_code      130 non-null    object 
 14  accessibility   130 non-null    object 
 15  price           130 non-null    object 
 16  url             14 non-null     object 
 17  Rambience       130 non-null    obj

In [22]:
# remove the "fax" and "url" column since they are mostly null
places_geo_df.drop(columns=['fax', 'url'], inplace=True)

#### Concatenate the restaurant dfs into a single df

In [23]:
from functools import reduce
places_df = reduce(lambda left,right: pd.merge(left, right, on='placeID', how='outer'), [places_accept_df, places_cuisine_df, places_parking_df, places_geo_df])
places_df.placeID = places_df.placeID.astype(str)
places_df.replace({np.nan: None}, inplace=True)
places_df

,placeID,payment,cuisine,parking_lot,latitude,longitude,the_geom_meter,name,address,city,...,zip,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
0,132002,MasterCard-Eurocard;Visa;American_Express;Dine...,Seafood,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,132012,Visa;American_Express;bank_debit_cards;Diners_...,French;Dutch-Belgian,public;street,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,132019,MasterCard-Eurocard;Visa;American_Express;Dine...,Dutch-Belgian,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,132023,American_Express;Japan_Credit_Bureau;bank_debi...,French;Dutch-Belgian,fee;public,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,132024,Visa;American_Express;bank_debit_cards;Diners_...,Seafood;Continental-European,street,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,133041,None,None,none,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
930,133042,None,None,none,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
931,134979,None,None,none,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
932,135108,None,None,none,22.1363,-100.934,0101000020957F00008FAE40D59E4B58C112C66046D597...,Potzocalli,Carretera Central Sn,San Luis Potosi,...,None,No_Alcohol_Served,none,informal,completely,low,familiar,f,closed,none


## User files

### usercuisine file

In [24]:
users_cuisine_df = pd.read_csv(data_folder / 'usercuisine.csv')
users_cuisine_df

,userID,Rcuisine
0,U1001,American
1,U1002,Mexican
2,U1003,Mexican
3,U1004,Bakery
4,U1004,Breakfast-Brunch
...,...,...
325,U1135,Southern
326,U1135,Dessert-Ice_Cream
327,U1136,Mexican
328,U1137,Mexican


In [25]:
users_cuisine_df.rename(columns={'Rcuisine': 'cuisine'}, inplace=True)
users_cuisine_df = users_cuisine_df.groupby('userID').agg({'cuisine': lambda x: '; '.join(x)}).reset_index()
users_cuisine_df

,userID,cuisine
0,U1001,American
1,U1002,Mexican
2,U1003,Mexican
3,U1004,Bakery; Breakfast-Brunch; Japanese; Contempora...
4,U1005,American
...,...,...
133,U1134,Mexican
134,U1135,Organic-Healthy; Steaks; Middle_Eastern; Medit...
135,U1136,Mexican
136,U1137,Mexican


### userpayment file

In [26]:
users_payment_df = pd.read_csv(data_folder / 'userpayment.csv')
users_payment_df

,userID,Upayment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash
4,U1004,bank_debit_cards
...,...,...
172,U1134,cash
173,U1135,cash
174,U1136,cash
175,U1137,cash


In [27]:
users_payment_df.rename(columns={'Upayment': 'payment'}, inplace=True)
users_payment_df = users_payment_df.groupby('userID').agg({'payment': lambda x: '; '.join(x)}).reset_index()
users_payment_df

,userID,payment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash; bank_debit_cards
4,U1005,cash
...,...,...
128,U1134,cash
129,U1135,cash
130,U1136,cash
131,U1137,cash


### userprofile file

In [28]:
users_profile_df = pd.read_csv(data_folder / 'userprofile.csv')
users_profile_df

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,U1134,22.149654,-100.998610,false,casual drinker,no preference,family,public,single,independent,1991,variety,hard-worker,Catholic,student,black,52,medium,1.65
134,U1135,22.170396,-100.949936,false,casual drinker,informal,family,on foot,single,kids,1988,variety,hunter-ostentatious,Catholic,student,purple,66,low,1.54
135,U1136,22.149607,-100.997235,true,social drinker,no preference,friends,car owner,single,independent,1990,retro,thrifty-protector,Catholic,student,black,50,low,1.60
136,U1137,22.144803,-100.944623,false,social drinker,formal,family,public,single,independent,1989,eco-friendly,hard-worker,Catholic,student,blue,72,low,1.78


In [29]:
users_df = reduce(lambda left,right: pd.merge(left, right, on='userID', how='outer'), [users_profile_df, users_cuisine_df, users_payment_df])
users_df.replace({np.nan: None}, inplace=True)
users_df

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,...,interest,personality,religion,activity,color,weight,budget,height,cuisine,payment
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,...,variety,thrifty-protector,none,student,black,69,medium,1.77,American,cash
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,...,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87,Mexican,cash
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,...,none,hard-worker,Catholic,student,blue,60,low,1.69,Mexican,cash
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,...,variety,hard-worker,none,professional,green,44,medium,1.53,Bakery; Breakfast-Brunch; Japanese; Contempora...,cash; bank_debit_cards
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,...,none,thrifty-protector,Catholic,student,black,65,medium,1.69,American,cash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,U1134,22.149654,-100.998610,false,casual drinker,no preference,family,public,single,independent,...,variety,hard-worker,Catholic,student,black,52,medium,1.65,Mexican,cash
134,U1135,22.170396,-100.949936,false,casual drinker,informal,family,on foot,single,kids,...,variety,hunter-ostentatious,Catholic,student,purple,66,low,1.54,Organic-Healthy; Steaks; Middle_Eastern; Medit...,cash
135,U1136,22.149607,-100.997235,true,social drinker,no preference,friends,car owner,single,independent,...,retro,thrifty-protector,Catholic,student,black,50,low,1.60,Mexican,cash
136,U1137,22.144803,-100.944623,false,social drinker,formal,family,public,single,independent,...,eco-friendly,hard-worker,Catholic,student,blue,72,low,1.78,Mexican,cash


## Ratings

In [30]:
ratings_df = pd.read_csv(data_folder / 'rating_final.csv')
ratings_df.placeID = ratings_df.placeID.astype(str)
ratings_df

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


# Populate data into Aito
Now that we have all the data loaded and processed, we will upload it to your Aito instance

## First, let's start with creating table schemas for each of the places, users, and ratings tables from their respective dataframes

### Infer the Aito Table schema from the places data frame

In [31]:
places_schema = AitoTableSchema.infer_from_pandas_data_frame(places_df)
print(places_schema.to_json_string(indent=2))

{
  "type": "table",
  "columns": {
    "placeID": {
      "type": "String",
      "nullable": false
    },
    "payment": {
      "type": "String",
      "nullable": true
    },
    "cuisine": {
      "type": "String",
      "nullable": true
    },
    "parking_lot": {
      "type": "String",
      "nullable": true
    },
    "latitude": {
      "type": "Decimal",
      "nullable": true
    },
    "longitude": {
      "type": "Decimal",
      "nullable": true
    },
    "the_geom_meter": {
      "type": "String",
      "nullable": true
    },
    "name": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "language",
        "language": "spanish",
        "useDefaultStopWords": false,
        "customStopWords": [],
        "customKeyWords": []
      }
    },
    "address": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "language",
        "language": "spanish",
        "useDefaultStopWords": false,
        "customSto

#### Aito didn't do well to detect some fields to be semicolon delimited. Let's adjust a bit

In [32]:
for field in ('payment', 'cuisine', 'parking_lot'):
    places_schema[field].data_type = AitoTextType()
    places_schema[field].analyzer = AitoDelimiterAnalyzerSchema(delimiter=';')
print(places_schema.to_json_string(indent=2))

{
  "type": "table",
  "columns": {
    "placeID": {
      "type": "String",
      "nullable": false
    },
    "payment": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "delimiter",
        "delimiter": ";",
        "trimWhitespace": true
      }
    },
    "cuisine": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "delimiter",
        "delimiter": ";",
        "trimWhitespace": true
      }
    },
    "parking_lot": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "delimiter",
        "delimiter": ";",
        "trimWhitespace": true
      }
    },
    "latitude": {
      "type": "Decimal",
      "nullable": true
    },
    "longitude": {
      "type": "Decimal",
      "nullable": true
    },
    "the_geom_meter": {
      "type": "String",
      "nullable": true
    },
    "name": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "language",
   

In [33]:
print(places_schema.to_json_string(indent=2))

{
  "type": "table",
  "columns": {
    "placeID": {
      "type": "String",
      "nullable": false
    },
    "payment": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "delimiter",
        "delimiter": ";",
        "trimWhitespace": true
      }
    },
    "cuisine": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "delimiter",
        "delimiter": ";",
        "trimWhitespace": true
      }
    },
    "parking_lot": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "delimiter",
        "delimiter": ";",
        "trimWhitespace": true
      }
    },
    "latitude": {
      "type": "Decimal",
      "nullable": true
    },
    "longitude": {
      "type": "Decimal",
      "nullable": true
    },
    "the_geom_meter": {
      "type": "String",
      "nullable": true
    },
    "name": {
      "type": "Text",
      "nullable": true,
      "analyzer": {
        "type": "language",
   

### Infer the Aito Table schema from the user data frame

In [34]:
users_schema = AitoTableSchema.infer_from_pandas_data_frame(users_df)
print(users_schema.to_json_string(indent=2))

{
  "type": "table",
  "columns": {
    "userID": {
      "type": "String",
      "nullable": false
    },
    "latitude": {
      "type": "Decimal",
      "nullable": false
    },
    "longitude": {
      "type": "Decimal",
      "nullable": false
    },
    "smoker": {
      "type": "String",
      "nullable": false
    },
    "drink_level": {
      "type": "Text",
      "nullable": false,
      "analyzer": "whitespace"
    },
    "dress_preference": {
      "type": "String",
      "nullable": false
    },
    "ambience": {
      "type": "String",
      "nullable": false
    },
    "transport": {
      "type": "String",
      "nullable": false
    },
    "marital_status": {
      "type": "String",
      "nullable": false
    },
    "hijos": {
      "type": "String",
      "nullable": false
    },
    "birth_year": {
      "type": "Int",
      "nullable": false
    },
    "interest": {
      "type": "String",
      "nullable": false
    },
    "personality": {
      "type": "Text",
  

#### Similarly, the users' cuisine and parking should be semicolon delimited

In [35]:
for field in ('payment', 'cuisine'):
    users_schema[field].data_type = AitoTextType()
    users_schema[field].analyzer = AitoDelimiterAnalyzerSchema(delimiter=';')

### Infer the Aito Table Schema from the ratings data frame

In [36]:
ratings_schema = AitoTableSchema.infer_from_pandas_data_frame(ratings_df)
print(ratings_schema.to_json_string(indent=2))

{
  "type": "table",
  "columns": {
    "userID": {
      "type": "String",
      "nullable": false
    },
    "placeID": {
      "type": "String",
      "nullable": false
    },
    "rating": {
      "type": "Int",
      "nullable": false
    },
    "food_rating": {
      "type": "Int",
      "nullable": false
    },
    "service_rating": {
      "type": "Int",
      "nullable": false
    }
  }
}


#### Link the ratings to the places and the users table

In [37]:
ratings_schema['placeID'].link = AitoColumnLinkSchema('places', 'placeID')
ratings_schema['userID'].link = AitoColumnLinkSchema('users', 'userID')

#### We now have our db schema:

In [38]:
db_schema = AitoDatabaseSchema(tables={'users': users_schema, 'places': places_schema, 'ratings': ratings_schema})

In [39]:
print(db_schema.to_json_string(indent=2))

{
  "schema": {
    "users": {
      "type": "table",
      "columns": {
        "userID": {
          "type": "String",
          "nullable": false
        },
        "latitude": {
          "type": "Decimal",
          "nullable": false
        },
        "longitude": {
          "type": "Decimal",
          "nullable": false
        },
        "smoker": {
          "type": "String",
          "nullable": false
        },
        "drink_level": {
          "type": "Text",
          "nullable": false,
          "analyzer": "whitespace"
        },
        "dress_preference": {
          "type": "String",
          "nullable": false
        },
        "ambience": {
          "type": "String",
          "nullable": false
        },
        "transport": {
          "type": "String",
          "nullable": false
        },
        "marital_status": {
          "type": "String",
          "nullable": false
        },
        "hijos": {
          "type": "String",
          "nullable": false


## Finally, set up the credentials and upload the data into Aito

In [41]:
# Fill in your Aito instance credentials
AITO_INSTANCE_URL = 'https://team1junction.aito.app'
AITO_API_KEY = 'l9uG4dc5ha7BoqBcgT58S2Z7G7uq75ES9w6scxTM'

In [42]:
client = AitoClient(instance_url=AITO_INSTANCE_URL, api_key=AITO_API_KEY)

In [43]:
aito_api.create_database(client=client, schema=db_schema)

In [44]:
aito_api.upload_entries(client=client, table_name='places', entries=places_df.to_dict(orient='records'))
aito_api.upload_entries(client=client, table_name='users', entries=users_df.to_dict(orient='records'))
aito_api.upload_entries(client=client, table_name='ratings', entries=ratings_df.to_dict(orient='records'))

# Now, lets try some queries

## Search for places that match an user's criteria

Let's assumme that this user like Mexican cuisine and would like to pay by cash

In [45]:
query = {
    "from": "places",
    "where": {
        "cuisine": "Mexican",
        "payment": "cash"
    }
}
res = aito_api.generic_query(client=client, query=query)
print(res.to_json_string(indent=2))

{
  "offset": 0,
  "total": 147,
  "hits": [
    {
      "cuisine": "Mexican",
      "parking_lot": "none",
      "payment": "cash",
      "placeID": "132578"
    },
    {
      "cuisine": "Mexican",
      "parking_lot": "none",
      "payment": "cash",
      "placeID": "132580"
    },
    {
      "Rambience": "familiar",
      "accessibility": "completely",
      "alcohol": "No_Alcohol_Served",
      "area": "closed",
      "cuisine": "Mexican",
      "dress_code": "informal",
      "franchise": "t",
      "latitude": 23.752364800000002,
      "longitude": -99.16528790000001,
      "name": "Gorditas Dona Tota",
      "other_services": "none",
      "parking_lot": "yes",
      "payment": "cash",
      "placeID": "132584",
      "price": "medium",
      "smoking_area": "not permitted",
      "the_geom_meter": "0101000020957F000048C65BA7EF8157C177F2344D664E4941"
    },
    {
      "cuisine": "Mexican",
      "parking_lot": "public",
      "payment": "cash",
      "placeID": "132588"
    

#### Pretty good but we want to return better results

## This time, we would like to suggest a restaurant to an user based on the previous ratings

In [61]:
rec_query = {
    "from": "ratings",
    "where": {
        "userID": {
            "$and": [
                {"userID": "U1022"},
                {"userID": "U1026"}
            ]
        },
        "placeID": {
            "price": "high"
        }
    },
    "recommend": "placeID",
    "goal": {"rating": 2},
    "limit": 5
}

In [62]:
res = aito_api.recommend(client=client, query=rec_query)
res.json

{'offset': 0,
 'total': 25,
 'hits': [{'$p': 0.9244336662379592,
   'Rambience': 'familiar',
   'accessibility': 'no_accessibility',
   'address': 'Ricardo Linares 107',
   'alcohol': 'Wine-Beer',
   'area': 'closed',
   'city': 'Cuernavaca',
   'country': 'Mexico',
   'cuisine': 'International',
   'dress_code': 'formal',
   'franchise': 'f',
   'latitude': 18.928798,
   'longitude': -99.239513,
   'name': 'Restaurant Las Mananitas',
   'other_services': 'none',
   'parking_lot': 'yes',
   'placeID': '134986',
   'price': 'high',
   'smoking_area': 'none',
   'state': 'Morelos',
   'the_geom_meter': '0101000020957F00002A0D05E2D96D5AC1AB058CB1EC564A41'},
  {'$p': 0.8847367370604847,
   'Rambience': 'familiar',
   'accessibility': 'no_accessibility',
   'address': 'Venustiano Carranza 560 Centro',
   'alcohol': 'Wine-Beer',
   'area': 'closed',
   'city': 'San Luis Potosi',
   'country': 'Mexico',
   'dress_code': 'casual',
   'franchise': 'f',
   'latitude': 22.151189000000002,
   'lon

## This is just a scratch on the surface. Now it's your turn to explore Aito's capabilities and make the best restaurant recommendation app